In [1]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [ ]:
model_version = 5
fng_feature_group_version = 1
solana_feature_group_version = 1
bitcoin_feature_group_version = 1

In [3]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

1736078344

In [4]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

2025-01-05 13:59:04,507 INFO: Initializing external client
2025-01-05 13:59:04,507 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 13:59:06,823 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [5]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


1732622344.0

In [6]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=solana_feature_group_version
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.69s) 


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1732665600,230.68,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184
1,1732752000,242.26,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987
2,1732838400,237.59,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213
3,1732924800,243.47,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804
4,1733011200,237.75,239.84,234.50,237.00,245637.142977,113485.713256,132151.429722


In [7]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=bitcoin_feature_group_version
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.72s) 


,timestamp,open
0,1732665600,91914.5
1,1732752000,95952.9
2,1732838400,95637.8
3,1732924800,97478.7
4,1733011200,96456.2


In [8]:
fng_feature_group = fs.get_feature_group(
    'f_n_g_index',
    version=fng_feature_group_version
)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['fng_classification'] = fng_batch_data['fng_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.58s) 


,fng_value,fng_classification,timestamp
0,75,3,1.732666e+09
1,77,4,1.732752e+09
2,78,4,1.732838e+09
3,84,4,1.732925e+09
4,81,4,1.733011e+09


In [9]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=model_version,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


/var/folders/rh/xp6gtwpn19s_8lqdxq8sdl0r0000gn/T/6e8580f9-4017-49a2-a343-2bce71dd2483/solana_price_daily_model_10d/4


<Sequential name=sequential, built=True>

In [10]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

,high,low,close,volume,volume_buy,volume_sell,open_bitcoin,fng_value,fng_classification,open_solana
0,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184,91914.5,75,3,230.68
1,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987,95952.9,77,4,242.26
2,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213,95637.8,78,4,237.59
3,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804,97478.7,84,4,243.47
4,239.84,234.50,237.00,245637.142977,113485.713256,132151.429722,96456.2,81,4,237.75


In [11]:
model_workspace = "solana_price_10d_scalers/" 
scaler_labels = joblib.load(model_workspace+'scaler_labels.pkl')
scaler_features = joblib.load(model_workspace+'scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations


In [12]:
all_data.columns

Index(['high', 'low', 'close', 'volume', 'volume_buy', 'volume_sell',
       'open_bitcoin', 'fng_value', 'fng_classification', 'open_solana'],
      dtype='object')

In [13]:
predictions = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


In [14]:
predictions

array([[0.7558749 , 0.78590834, 0.764528  , 0.75248384, 0.7892312 ,
        0.73513407, 0.737531  , 0.7450325 , 0.7272974 , 0.7641749 ]],
      dtype=float32)

In [15]:
predictions_unscaled =  scaler_labels.inverse_transform(predictions)
predictions_unscaled


array([[197.87308, 205.35262, 200.02806, 197.02858, 206.18015, 192.7078 ,
        193.30473, 195.1729 , 190.75616, 199.94012]], dtype=float32)

In [16]:

timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions[0]))]

In [17]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=4,
    primary_key=['date']
)

In [18]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions_unscaled[0]
result

,timestamp,open_solana
0,1736164744,197.873077
1,1736251144,205.352615
2,1736337544,200.028061
3,1736423944,197.028580
4,1736510344,206.180145
5,1736596744,192.707794
6,1736683144,193.304733
7,1736769544,195.172897
8,1736855944,190.756165
9,1736942344,199.940125


In [19]:
result['date'] = pd.to_datetime(result['timestamp'], unit='s')
result['date'] = result['date'].dt.normalize()
result= result.sort_values(by=['date'])

result.drop(columns=["timestamp"], inplace=True)

result

,open_solana,date
0,197.873077,2025-01-06
1,205.352615,2025-01-07
2,200.028061,2025-01-08
3,197.028580,2025-01-09
4,206.180145,2025-01-10
5,192.707794,2025-01-11
6,193.304733,2025-01-12
7,195.172897,2025-01-13
8,190.756165,2025-01-14
9,199.940125,2025-01-15


In [20]:
#adding dummy records to test the plots
# New record
new_record = {"date": pd.to_datetime("2025-01-05"), "open_solana": 202}
new_record_1 = {"date": pd.to_datetime("2025-01-04"), "open_solana": 200}
new_record_2 = {"date": pd.to_datetime("2025-01-03"), "open_solana": 198}


# Append the new record
new_record_df = pd.DataFrame([new_record])
new_record_df_1 = pd.DataFrame([new_record_1])
new_record_df_2 = pd.DataFrame([new_record_2])
result = pd.concat([result, new_record_df,new_record_df_1,new_record_df_2], ignore_index=True)
result= result.sort_values(by=['date'])

result

,open_solana,date
12,198.000000,2025-01-03
11,200.000000,2025-01-04
10,202.000000,2025-01-05
0,197.873077,2025-01-06
1,205.352615,2025-01-07
2,200.028061,2025-01-08
3,197.028580,2025-01-09
4,206.180145,2025-01-10
5,192.707794,2025-01-11
6,193.304733,2025-01-12


In [21]:
#this is plane and we may have duplicates
#monitor_fg.insert(result)

In [22]:
#this insert prevents duplicates
monitor_fg.insert(
    result,
    # overwrite=False,  # Do not overwrite all data
    operation="upsert",  # Use upsert to handle duplicates based on the primary key
    #storage=None  # Write to both online and offline storage
)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1160346/fs/1151049/fg/1394563


Uploading Dataframe: 100.00% |██████████| Rows 13/13 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_predictions_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1160346/jobs/named/solana_predictions_4_offline_fg_materialization/executions


(Job('solana_predictions_4_offline_fg_materialization', 'SPARK'), None)